In [ ]:
import keras
from keras import regularizers
import keras.backend as K
import tensorflow as tf
import numpy as np

import utils
import loggingreporter 

cfg = {}
cfg['SGD_BATCHSIZE']    = 128
cfg['SGD_LEARNINGRATE'] = 0.001
cfg['NUM_EPOCHS']       = 5000
cfg['FULL_MI']          = True

# cfg['ACTIVATION'] = 'tanh'
cfg['ACTIVATION'] = 'relu'

# How many hidden neurons to put into each of the layers
cfg['LAYER_DIMS'] = [10,7,5,4,3] # original IB network
ARCH_NAME =  '-'.join(map(str,cfg['LAYER_DIMS']))

# No validation data is created. Only TRAIN and TEST
trn, tst = utils.get_IB_data('2017_12_21_16_51_3_275766')

# Where to save activation and weights data
cfg['SAVE_DIR'] = 'rawdata/' + cfg['ACTIVATION'] + '_' + ARCH_NAME 

In [ ]:
input_layer = keras.layers.Input((trn.X.shape[1],))
clayer = input_layer
# Truncated norm initalisation is recommended for neural network weights, but why devided by sqrt(layer dimmension)?
for n in cfg['LAYER_DIMS']:
    clayer = keras.layers.Dense(n, 
                                activation=cfg['ACTIVATION'],
                                kernel_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=1/np.sqrt(float(n)), seed=10),
                                bias_initializer='zeros'
#                                 kernel_regularizer=regularizers.l2(0.01) # add L2 norm regularization
                               )(clayer)
#     clayer = keras.layers.Dropout(0.1)(clayer)
output_layer = keras.layers.Dense(trn.nb_classes, activation='softmax')(clayer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)
# Tune optimizer & learning rate as well? perhaps too much..
# optimizer = keras.optimizers.TFOptimizer(tf.train.AdamOptimizer(learning_rate=cfg['SGD_LEARNINGRATE']))

# sgd = keras.optimizers.SGD(lr=cfg['SGD_LEARNINGRATE'], decay=1e-6, momentum=0.9, nesterov=True)
# This is to study how optimizer with gradient clipping behaves in information plane
sgd = keras.optimizers.SGD(lr=cfg['SGD_LEARNINGRATE'], momentum=0.9, nesterov=True, clipnorm=1.0, clipvalue=1)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['binary_accuracy'])

In [ ]:
def do_report(epoch):
    # Only log activity for some epochs.  Mainly this is to make things run faster.
    if epoch < 20:       # Log for all first 20 epochs
        return True
    elif epoch < 100:    # Then for every 5th epoch
        return (epoch % 5 == 0)
    elif epoch < 2000:    # Then every 10th
        return (epoch % 20 == 0)
    else:                # Then every 100th
        return (epoch % 100 == 0)
    
# This is the callback function used to save parameters used to calculate MI. Note that MI is not calculated here.
reporter = loggingreporter.LoggingReporter(cfg=cfg, 
                                          trn=trn, 
                                          tst=tst, 
                                          do_save_func=do_report)
r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 2, 
              batch_size = cfg['SGD_BATCHSIZE'],
              epochs     = cfg['NUM_EPOCHS'],
              validation_split = 0.2,
              shuffle = True,
              # validation_data=(tst.X, tst.Y),
              callbacks  = [reporter,])